In [1]:
# pneumonia_kaggle_repro.py
# Ready-to-run: Full reproduction pipeline for
# "Pneumonia Detection from Chest X-Ray Images Using Deep Learning and Transfer Learning for Imbalanced Datasets"
#
# Intended for Kaggle (attach dataset 'paultimothymooney/chest-xray-pneumonia' in notebook "Add data")
#
# Requirements (on Kaggle): torch, torchvision, timm, scikit-learn, pandas, matplotlib, seaborn, tqdm
# Kaggle kernels already include most packages; if missing use pip install in a cell.

# ---------------------------
# 0) Imports & Environment
# ---------------------------
import os
import random
import time
import copy
from pathlib import Path
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms, models
import timm

from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, roc_curve, auc, precision_recall_curve)
from tqdm.auto import tqdm

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:


# ---------------------------
# 1) Configuration
# ---------------------------
class CFG:
    seed = 42
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    img_size = 224
    batch_size = 32
    val_batch_size = 64
    epochs = 12           # paper uses multiple epochs; adjust if you need a longer run
    lr_head = 1e-3        # LR for head-only fine-tune
    lr_full = 2e-4        # LR for full fine-tune
    out_dir = Path('./outputs')
    data_root = Path('/kaggle/input/chest-xray-pneumonia/chest_xray')  # Fixed: direct path to chest_xray
    num_workers = 2
    use_timm_vit = True   # requires timm (present on Kaggle)
    pretrained = True
    save_every = 1
    early_stop_patience = 4

CFG.out_dir.mkdir(parents=True, exist_ok=True)

# reproducibility
def seed_everything(seed=CFG.seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
seed_everything()

print("Device:", CFG.device)
print("Data root:", CFG.data_root)

Device: cuda
Data root: /kaggle/input/chest-xray-pneumonia/chest_xray


In [3]:


# ---------------------------
# 2) Dataset helpers - FIXED DATA LEAKAGE
# ---------------------------
def list_images(folder):
    exts = ['*.jpeg','*.jpg','*.png','*.JPEG','*.JPG','*.PNG']
    files = []
    for ext in exts:
        files += list(Path(folder).rglob(ext))
    return sorted([str(x) for x in files])

def load_kaggle_split(data_root):
    """Load dataset splits with proper validation set handling"""
    chest_folder = Path(data_root)
    if not chest_folder.exists():
        raise FileNotFoundError(f"Expected dataset at {chest_folder}. On Kaggle, attach dataset 'paultimothymooney/chest-xray-pneumonia'.")
    
    def read_split(split):
        out=[]
        split_dir = chest_folder / split
        if not split_dir.exists():
            return []
        for cls_dir in split_dir.iterdir():
            if not cls_dir.is_dir(): 
                continue
            clsname = cls_dir.name.lower()
            label = 1 if 'pneumonia' in clsname else 0
            for p in list_images(cls_dir):
                out.append((p, label))
        return out
    
    train = read_split('train')
    val = read_split('val')
    test = read_split('test')
    
    # FIX FOR DATA LEAKAGE: The original validation set is too small (16 images)
    # Combine train + val and create proper 80/10/10 split
    if len(val) < 100:  # If validation set is too small
        print(f"Warning: Validation set too small ({len(val)} samples). Creating proper train/val split from training data.")
        all_train_val = train + val
        random.shuffle(all_train_val)
        
        train_ratio = 0.8
        val_ratio = 0.1
        # test_ratio = 0.1 (test set remains separate)
        
        train_count = int(len(all_train_val) * train_ratio)
        val_count = int(len(all_train_val) * val_ratio)
        
        train = all_train_val[:train_count]
        val = all_train_val[train_count:train_count + val_count]
        # The original test set remains unchanged
    
    return train, val, test

train_samples, val_samples, test_samples = load_kaggle_split(CFG.data_root)
print("Counts - train:", len(train_samples), "val:", len(val_samples), "test:", len(test_samples))
print("Class distribution - Train:", Counter([y for _, y in train_samples]))
print("Class distribution - Val:", Counter([y for _, y in val_samples]))
print("Class distribution - Test:", Counter([y for _, y in test_samples]))

Counts - train: 4185 val: 523 test: 624
Class distribution - Train: Counter({1: 3090, 0: 1095})
Class distribution - Val: Counter({1: 394, 0: 129})
Class distribution - Test: Counter({1: 390, 0: 234})


In [4]:


# ---------------------------
# 3) Dataset class + transforms - FIXED AUGMENTATION
# ---------------------------
# Strong augmentation for Mean Teacher student
strong_transform = transforms.Compose([
    transforms.Resize((CFG.img_size, CFG.img_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(CFG.img_size, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_transform = transforms.Compose([
    transforms.Resize((CFG.img_size, CFG.img_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.12, contrast=0.12),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((CFG.img_size, CFG.img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class ChestXrayDataset(Dataset):
    def __init__(self, samples, transform=None, strong_transform=None):
        self.samples = samples
        self.transform = transform
        self.strong_transform = strong_transform
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        p, y = self.samples[idx]
        try:
            img = Image.open(p).convert('RGB')
        except Exception as e:
            print(f"Error loading image {p}: {e}")
            # Return a blank image as fallback
            img = Image.new('RGB', (CFG.img_size, CFG.img_size), color='black')
        
        if self.strong_transform:
            # For Mean Teacher: return both weak and strong augmented versions
            img_weak = self.transform(img) if self.transform else img
            img_strong = self.strong_transform(img)
            return img_weak, img_strong, torch.tensor(y, dtype=torch.long)
        elif self.transform:
            img = self.transform(img)
            return img, torch.tensor(y, dtype=torch.long)
        else:
            return img, torch.tensor(y, dtype=torch.long)

In [5]:


# ---------------------------
# 4) Imbalance handling utilities
# ---------------------------
def compute_class_weights(samples):
    cnt = Counter([y for _,y in samples])
    total = sum(cnt.values())
    weights = {cls: total / (len(cnt) * count) for cls, count in cnt.items()}
    ordered = [weights[i] for i in sorted(weights.keys())]
    return torch.tensor(ordered, dtype=torch.float).to(CFG.device), weights

def make_weighted_sampler(samples):
    _, weights_map = compute_class_weights(samples)
    sample_weights = [weights_map[y] for _, y in samples]
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
    return sampler

# Focal loss implementation
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.weight = weight
        self.reduction = reduction
        self.ce = nn.CrossEntropyLoss(weight=weight, reduction='none')
        
    def forward(self, logits, target):
        loss = self.ce(logits, target)
        p_t = torch.exp(-loss)
        focal = ((1 - p_t) ** self.gamma) * loss
        if self.reduction == 'mean':
            return focal.mean()
        elif self.reduction == 'sum':
            return focal.sum()
        else:
            return focal

In [6]:


# ---------------------------
# 5) Model factories (VGG16, ResNet50, ViT)
# ---------------------------
def get_model(name='resnet50', num_classes=2, pretrained=True):
    name = name.lower()
    if name == 'vgg16':
        model = models.vgg16(pretrained=pretrained)
        in_f = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_f, num_classes)
    elif name == 'resnet50':
        model = models.resnet50(pretrained=pretrained)
        in_f = model.fc.in_features
        model.fc = nn.Linear(in_f, num_classes)
    elif name == 'vit':
        model = timm.create_model('vit_base_patch16_224', pretrained=pretrained, num_classes=num_classes)
    else:
        raise ValueError("Unknown model: "+name)
    return model


In [7]:

# ---------------------------
# 6) Metrics & plotting utilities
# ---------------------------
def compute_metrics(y_true, y_pred, probs=None):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auc_score = float('nan')
    if probs is not None and len(set(y_true)) > 1:
        try:
            auc_score = roc_auc_score(y_true, probs)
        except:
            auc_score = float('nan')
    return {'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1, 'auc': auc_score}

def plot_training_curves(history, prefix):
    epochs = range(1, len(history['train_loss']) + 1)
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].plot(epochs, history['train_loss'], label='train_loss')
    axes[0].plot(epochs, history['val_loss'], label='val_loss')
    axes[0].set_title('Loss')
    axes[0].legend()
    axes[1].plot(epochs, history['val_f1'], label='val_f1')
    axes[1].plot(epochs, history['val_auc'], label='val_auc')
    axes[1].set_title('Validation metrics')
    axes[1].legend()
    plt.tight_layout()
    outp = CFG.out_dir / f'{prefix}_training.png'
    plt.savefig(outp, dpi=150)
    plt.close()

def plot_roc_curve(y_true, probs, prefix):
    fpr, tpr, _ = roc_curve(y_true, probs)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(5, 4))
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
    plt.plot([0, 1], [0, 1], '--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.tight_layout()
    outp = CFG.out_dir / f'{prefix}_roc.png'
    plt.savefig(outp, dpi=150)
    plt.close()

def plot_confusion_matrix(y_true, preds, prefix):
    cm = confusion_matrix(y_true, preds)
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    outp = CFG.out_dir / f'{prefix}_confmat.png'
    plt.savefig(outp, dpi=150)
    plt.close()

In [8]:


# ---------------------------
# 7) Training & evaluation loops - FIXED ISSUES
# ---------------------------
def evaluate_model(model, loader, criterion=None):
    model.eval()
    ys, preds, probs = [], [], []
    total_loss = 0.0
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(CFG.device)
            yb = yb.to(CFG.device)
            out = model(xb)
            
            if criterion is not None:
                loss = criterion(out, yb)
                total_loss += loss.item() * xb.size(0)
                
            prob = torch.softmax(out, dim=1)[:, 1].cpu().numpy()
            pred = out.argmax(dim=1).cpu().numpy()
            ys.extend(yb.cpu().numpy().tolist())
            preds.extend(pred.tolist())
            probs.extend(prob.tolist())
    
    avg_loss = total_loss / len(loader.dataset) if criterion is not None else 0.0
    return ys, preds, probs, avg_loss

def train_and_evaluate(model_name,
                       train_samples, val_samples, test_samples,
                       imbalance_strategy='weighted_sampler',
                       transfer_strategy='head_then_full',
                       use_mean_teacher=False,
                       unlabeled_samples=None,
                       run_name='run'):
    """
    Fixed training function with proper validation loss calculation and Mean Teacher implementation
    """

    # Prepare datasets & loaders
    if use_mean_teacher:
        train_ds = ChestXrayDataset(train_samples, transform=val_transform, strong_transform=strong_transform)
    else:
        train_ds = ChestXrayDataset(train_samples, transform=train_transform)
        
    val_ds = ChestXrayDataset(val_samples, transform=val_transform)
    test_ds = ChestXrayDataset(test_samples, transform=val_transform)

    if imbalance_strategy == 'weighted_sampler':
        sampler = make_weighted_sampler(train_samples)
        train_loader = DataLoader(train_ds, batch_size=CFG.batch_size, sampler=sampler, 
                                 num_workers=CFG.num_workers, pin_memory=True)
    else:
        train_loader = DataLoader(train_ds, batch_size=CFG.batch_size, shuffle=True, 
                                 num_workers=CFG.num_workers, pin_memory=True)

    val_loader = DataLoader(val_ds, batch_size=CFG.val_batch_size, shuffle=False, 
                           num_workers=CFG.num_workers, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=CFG.val_batch_size, shuffle=False, 
                            num_workers=CFG.num_workers, pin_memory=True)

    # model
    model = get_model(model_name, num_classes=2, pretrained=CFG.pretrained).to(CFG.device)

    # teacher (Mean Teacher) - FIXED IMPLEMENTATION
    teacher = None
    if use_mean_teacher:
        teacher = copy.deepcopy(model)
        for p in teacher.parameters():
            p.requires_grad = False
        teacher.to(CFG.device)
        teacher.eval()

    # loss
    class_weight_tensor, class_map = compute_class_weights(train_samples)
    if imbalance_strategy == 'class_weight':
        criterion = nn.CrossEntropyLoss(weight=class_weight_tensor)
    elif imbalance_strategy == 'focal':
        criterion = FocalLoss(gamma=2.0, weight=class_weight_tensor)
    else:
        criterion = nn.CrossEntropyLoss()

    # optimizer & schedulers
    if transfer_strategy == 'head_then_full':
        def freeze_backbone(m, name):
            if name == 'vgg16':
                for param in m.features.parameters():
                    param.requires_grad = False
            elif name == 'resnet50':
                for n, p in m.named_parameters():
                    if not n.startswith('fc'):
                        p.requires_grad = False
            elif name == 'vit':
                for n, p in m.named_parameters():
                    if not n.startswith('head'):
                        p.requires_grad = False
        freeze_backbone(model, model_name)
        optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=CFG.lr_head)
    else:
        optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr_full)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)

    # training loop variables
    history = {'train_loss': [], 'val_loss': [], 'val_f1': [], 'val_auc': []}
    best_val_f1 = -1.0
    epochs_no_improve = 0
    best_epoch = 0

    # helper to update EMA teacher - FIXED
    def update_ema(student, teacher, alpha=0.99):
        with torch.no_grad():
            for sp, tp in zip(student.parameters(), teacher.parameters()):
                tp.data.mul_(alpha).add_(sp.data * (1 - alpha))

    # Unlabeled loader for mean teacher consistency
    unlabeled_loader = None
    unl_iter = None
    if use_mean_teacher and unlabeled_samples:
        unl_ds = ChestXrayDataset(unlabeled_samples, transform=val_transform, strong_transform=strong_transform)
        unlabeled_loader = DataLoader(unl_ds, batch_size=CFG.batch_size, shuffle=True, 
                                    num_workers=CFG.num_workers, pin_memory=True)
        unl_iter = iter(unlabeled_loader)

    # training
    for epoch in range(1, CFG.epochs + 1):
        if epochs_no_improve >= CFG.early_stop_patience:
            print(f"Early stopping triggered at epoch {epoch}")
            break
            
        model.train()
        running_loss = 0.0
        n = 0
        pbar = tqdm(train_loader, desc=f"{run_name} {model_name} E{epoch}/{CFG.epochs} [{imbalance_strategy}]")
        
        for batch in pbar:
            if use_mean_teacher:
                xb_weak, xb_strong, yb = batch
                xb_weak = xb_weak.to(CFG.device)
                xb_strong = xb_strong.to(CFG.device)
                yb = yb.to(CFG.device)
            else:
                xb, yb = batch
                xb = xb.to(CFG.device)
                yb = yb.to(CFG.device)
                
            optimizer.zero_grad()
            
            if use_mean_teacher:
                # Student forward with strong augmentation
                logits_s = model(xb_strong)
                supervised_loss = criterion(logits_s, yb)
                
                # Teacher forward with weak augmentation
                with torch.no_grad():
                    logits_t = teacher(xb_weak)
                    probs_t = torch.softmax(logits_t, dim=1)
                
                # Consistency loss
                probs_s = torch.softmax(logits_s, dim=1)
                consistency_loss = nn.MSELoss()(probs_s, probs_t)
                
                # Combined loss
                lambda_consistency = 1.0
                loss = supervised_loss + lambda_consistency * consistency_loss
            else:
                logits = model(xb)
                loss = criterion(logits, yb)

            loss.backward()
            optimizer.step()

            if use_mean_teacher and teacher is not None:
                update_ema(model, teacher, alpha=0.99)

            running_loss += loss.item() * (xb_strong.size(0) if use_mean_teacher else xb.size(0))
            n += (xb_strong.size(0) if use_mean_teacher else xb.size(0))
            pbar.set_postfix(loss=(running_loss / n))

        train_loss = running_loss / n

        # Validation evaluation - FIXED: Now computes actual validation loss
        ys_val, preds_val, probs_val, val_loss = evaluate_model(model, val_loader, criterion)
        val_metrics = compute_metrics(ys_val, preds_val, probs_val)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_f1'].append(val_metrics['f1'])
        history['val_auc'].append(val_metrics['auc'])

        print(f"Epoch {epoch} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | val_f1={val_metrics['f1']:.4f} | val_auc={val_metrics['auc']:.4f}")

        scheduler.step(val_metrics['f1'])

        # Save best
        if val_metrics['f1'] > best_val_f1:
            best_val_f1 = val_metrics['f1']
            epochs_no_improve = 0
            best_epoch = epoch
            torch.save(model.state_dict(), CFG.out_dir / f"{run_name}_{model_name}_{imbalance_strategy}_best.pth")
        else:
            epochs_no_improve += 1

        # If we used head_then_full, after a few epochs unfreeze backbone and switch optimizer to lower LR
        if transfer_strategy == 'head_then_full' and epoch == 3:
            for p in model.parameters():
                p.requires_grad = True
            optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr_full)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)
            print("Unfroze backbone; switched to full fine-tune with lr", CFG.lr_full)

    # Load best model
    best_model_path = CFG.out_dir / f"{run_name}_{model_name}_{imbalance_strategy}_best.pth"
    if best_model_path.exists():
        model.load_state_dict(torch.load(best_model_path))
        print(f"Loaded best model from epoch {best_epoch}")
    else:
        print("Warning: Best model not found, using final model")
    
    # Final evaluation on test set
    ys_test, preds_test, probs_test, _ = evaluate_model(model, test_loader)
    test_metrics = compute_metrics(ys_test, preds_test, probs_test)

    # save logs/plots
    prefix = f"{run_name}_{model_name}_{imbalance_strategy}"
    pd.DataFrame([{
        'epoch': i + 1, 
        'train_loss': history['train_loss'][i] if i < len(history['train_loss']) else None,
        'val_loss': history['val_loss'][i] if i < len(history['val_loss']) else None,
        'val_f1': history['val_f1'][i] if i < len(history['val_f1']) else None,
        'val_auc': history['val_auc'][i] if i < len(history['val_auc']) else None
    } for i in range(len(history['train_loss']))]).to_csv(CFG.out_dir / f"{prefix}_history.csv", index=False)

    plot_training_curves(history, prefix)
    if len(set(ys_test)) > 1:
        plot_roc_curve(ys_test, probs_test, prefix)
    plot_confusion_matrix(ys_test, preds_test, prefix)

    # save test metrics
    res = {
        'model': model_name, 
        'imbalance_strategy': imbalance_strategy,
        'test_acc': test_metrics['acc'], 
        'test_prec': test_metrics['prec'],
        'test_rec': test_metrics['rec'], 
        'test_f1': test_metrics['f1'], 
        'test_auc': test_metrics['auc']
    }
    pd.DataFrame([res]).to_csv(CFG.out_dir / f'{prefix}_test_metrics.csv', index=False)
    print("Test metrics:", res)
    return res

In [9]:


# ---------------------------
# 8) Experiment runner
# ---------------------------
def run_all_experiments(train_samples, val_samples, test_samples, unlabeled_samples=None, run_name='paper_repro'):
    models = ['vgg16', 'resnet50', 'vit']
    imbalance_strategies = ['none', 'weighted_sampler', 'class_weight', 'focal']
    transfer_strategy = 'head_then_full'
    results = []
    
    for m in models:
        for imb in imbalance_strategies:
            print("=" * 60)
            print(f"Running model={m}, imbalance={imb}")
            try:
                res = train_and_evaluate(
                    m, train_samples, val_samples, test_samples,
                    imbalance_strategy=imb,
                    transfer_strategy=transfer_strategy,
                    use_mean_teacher=False,
                    unlabeled_samples=unlabeled_samples,
                    run_name=run_name
                )
                results.append(res)
            except Exception as e:
                print(f"Error running {m} with {imb}: {e}")
                continue
                
    # Save combined results
    if results:
        df = pd.DataFrame(results)
        df.to_csv(CFG.out_dir / f'{run_name}_summary_results.csv', index=False)
        print("All experiments finished. Summary saved to", CFG.out_dir / f'{run_name}_summary_results.csv')
    else:
        df = pd.DataFrame()
        print("No results to save.")
        
    return df

In [10]:


# ---------------------------
# 9) Optional: Mean Teacher experiments
# ---------------------------
def run_mean_teacher_experiment(train_samples, val_samples, test_samples, unlabeled_samples, run_name='mean_teacher'):
    return train_and_evaluate('resnet50', train_samples, val_samples, test_samples,
                              imbalance_strategy='weighted_sampler',
                              transfer_strategy='head_then_full',
                              use_mean_teacher=True,
                              unlabeled_samples=unlabeled_samples,
                              run_name=run_name)

# ---------------------------
# 10) Utility: Create unlabeled pool
# ---------------------------
def create_unlabeled_pool_from_train(train_samples, fraction=0.5):
    n = int(len(train_samples) * fraction)
    random.shuffle(train_samples)
    return [(p, 0) for p, _ in train_samples[:n]]

# ---------------------------
# 11) Run the full reproduction
# ---------------------------
if __name__ == "__main__":
    # Quick checks
    if len(train_samples) == 0:
        raise RuntimeError("No training samples found. Make sure you've attached the Kaggle dataset and data path is correct.")

    # Option: perform a lightweight demo run for quick verification
    QUICK_DEMO = False  # Set to True for quick testing
    
    if QUICK_DEMO:
        print("Running in DEMO mode with smaller subsets...")
        train_sub = train_samples[:1000]  # Smaller subset for demo
        val_sub = val_samples[:200]
        test_sub = test_samples[:200]
        unl_sub = create_unlabeled_pool_from_train(train_sub, fraction=0.3)
        df = run_all_experiments(train_sub, val_sub, test_sub, unlabeled_samples=unl_sub, run_name='demo')
    else:
        print("Running FULL reproduction...")
        unlabeled_pool = None  # set if you have extra unlabeled images
        summary_df = run_all_experiments(train_samples, val_samples, test_samples, 
                                       unlabeled_samples=unlabeled_pool, run_name='paper_repro')

        # (Optional) Run a Mean Teacher experiment
        # unlabeled_pool = create_unlabeled_pool_from_train(train_samples, fraction=0.5)
        # mt_res = run_mean_teacher_experiment(train_samples, val_samples, test_samples, 
        #                                    unlabeled_pool, run_name='mean_teacher_experiment')

        if not summary_df.empty:
            print("Summary:\n", summary_df)
        else:
            print("No results to display.")

Running FULL reproduction...
Running model=vgg16, imbalance=none


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 187MB/s]  


paper_repro vgg16 E1/12 [none]:   0%|          | 0/131 [00:00<?, ?it/s]

Epoch 1 | train_loss=0.4591 | val_loss=2.9310 | val_f1=0.8156 | val_auc=0.9891


paper_repro vgg16 E2/12 [none]:   0%|          | 0/131 [00:00<?, ?it/s]

...

paper_repro vit E5/12 [focal]:   0%|          | 0/131 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d715a9023e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d715a9023e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 16

Epoch 5 | train_loss=0.1033 | val_loss=0.1674 | val_f1=0.3445 | val_auc=0.9787
Early stopping triggered at epoch 6
Loaded best model from epoch 1


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d715a9023e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d715a9023e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 16

Test metrics: {'model': 'vit', 'imbalance_strategy': 'focal', 'test_acc': 0.8477564102564102, 'test_prec': 0.8185745140388769, 'test_rec': 0.9717948717948718, 'test_f1': 0.88862837045721, 'test_auc': 0.9526079333771642}
All experiments finished. Summary saved to outputs/paper_repro_summary_results.csv
Summary:
        model imbalance_strategy  test_acc  test_prec  test_rec   test_f1  \
0      vgg16               none  0.822115   0.781818  0.992308  0.874576   
1      vgg16   weighted_sampler  0.788462   0.748077  0.997436  0.854945   
2      vgg16       class_weight  0.823718   0.790456  0.976923  0.873853   
3      vgg16              focal  0.865385   0.844595  0.961538  0.899281   
4   resnet50               none  0.764423   0.726257  1.000000  0.841424   
5   resnet50   weighted_sampler  0.863782   0.823779  0.994872  0.901278   
6   resnet50       class_weight  0.838141   0.797938  0.992308  0.884571   
7   resnet50              focal  0.852564   0.810417  0.997436  0.894253   
8  

In [11]:
import zipfile
from pathlib import Path

def zip_selected_files():
    outputs_dir = Path('/kaggle/working/outputs')
    zip_path = '/kaggle/working/outputs_essential.zip'
    
    include_extensions = {'.png', '.csv', '.jpg', '.jpeg'}
    
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in outputs_dir.rglob('*'):
            if file_path.is_file() and file_path.suffix.lower() in include_extensions:
                arcname = file_path.relative_to('/kaggle/working')
                zipf.write(file_path, arcname)
    
    print(f"Created essential files zip: {zip_path}")
    return zip_path

zip_selected_files()

Created essential files zip: /kaggle/working/outputs_essential.zip


'/kaggle/working/outputs_essential.zip'

In [12]:
import zipfile
import os
from pathlib import Path

def zip_outputs_folder():
    outputs_dir = Path('/kaggle/working/outputs')
    zip_path = '/kaggle/working/outputs_complete.zip'
    
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in outputs_dir.rglob('*'):
            if file_path.is_file():
                # Create relative path for zip
                arcname = file_path.relative_to('/kaggle/working')
                zipf.write(file_path, arcname)
    
    print(f"Created zip file: {zip_path}")
    print(f"Size: {os.path.getsize(zip_path) / (1024*1024):.2f} MB")
    
    # List what's included
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        file_list = zipf.namelist()
        print(f"\nContains {len(file_list)} files:")
        for f in sorted(file_list)[:10]:  # Show first 10 files
            print(f"  - {f}")
        if len(file_list) > 10:
            print(f"  ... and {len(file_list) - 10} more files")
    
    return zip_path

# Create the zip
zip_file = zip_outputs_folder()

Created zip file: /kaggle/working/outputs_complete.zip
Size: 3453.35 MB

Contains 73 files:
  - outputs/paper_repro_resnet50_class_weight_best.pth
  - outputs/paper_repro_resnet50_class_weight_confmat.png
  - outputs/paper_repro_resnet50_class_weight_history.csv
  - outputs/paper_repro_resnet50_class_weight_roc.png
  - outputs/paper_repro_resnet50_class_weight_test_metrics.csv
  - outputs/paper_repro_resnet50_class_weight_training.png
  - outputs/paper_repro_resnet50_focal_best.pth
  - outputs/paper_repro_resnet50_focal_confmat.png
  - outputs/paper_repro_resnet50_focal_history.csv
  - outputs/paper_repro_resnet50_focal_roc.png
  ... and 63 more files
